In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train['HF1'] = train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Fire_Points']
train['HF2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Fire_Points'])
train['HR1'] = abs(train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Roadways'])
train['HR2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Roadways'])
train['FR1'] = abs(train['Horizontal_Distance_To_Fire_Points']+train['Horizontal_Distance_To_Roadways'])
train['FR2'] = abs(train['Horizontal_Distance_To_Fire_Points']-train['Horizontal_Distance_To_Roadways'])
train['ele_vert'] = train.Elevation-train.Vertical_Distance_To_Hydrology

train['slope_hyd'] = (train['Horizontal_Distance_To_Hydrology']**2+train['Vertical_Distance_To_Hydrology']**2)**0.5
train.slope_hyd=train.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any

#Mean distance to Amenities 
train['Mean_Amenities']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology + train.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
train['Mean_Fire_Hyd']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology) / 2

Feature Creation for test

In [4]:
test['HF1'] = test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Fire_Points']
test['HF2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Fire_Points'])
test['HR1'] = abs(test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Roadways'])
test['HR2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Roadways'])
test['FR1'] = abs(test['Horizontal_Distance_To_Fire_Points']+test['Horizontal_Distance_To_Roadways'])
test['FR2'] = abs(test['Horizontal_Distance_To_Fire_Points']-test['Horizontal_Distance_To_Roadways'])
test['ele_vert'] = test.Elevation-test.Vertical_Distance_To_Hydrology

test['slope_hyd'] = (test['Horizontal_Distance_To_Hydrology']**2+test['Vertical_Distance_To_Hydrology']**2)**0.5
test.slope_hyd=test.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any

#Mean distance to Amenities 
test['Mean_Amenities']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology + test.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
test['Mean_Fire_Hyd']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology) / 2

In [5]:
X_train = train.drop(['Id','Cover_Type'],1)
Y_train = train['Cover_Type']
X_test = test.drop(['Id'],1)

In [6]:
scaler = StandardScaler()
X_train_tf = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index, columns=X_train.columns)
X_test_tf = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns=X_test.columns)

In [7]:
X_train_tf.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,HF1,HF2,HR1,HR2,FR1,FR2,ele_vert,slope_hyd,Mean_Amenities,Mean_Fire_Hyd
0,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,4.334805,...,4.165194,4.390517,-0.848236,-0.959357,1.702247,5.505993,-0.246574,0.102333,1.673476,4.165194
1,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,4.285710,...,4.078395,4.383090,-0.968254,-1.016605,1.619137,5.580595,-0.246574,-0.110745,1.571055,4.078395
2,0.130912,-0.160577,-0.887379,0.194243,0.227369,1.106379,0.696843,0.834797,-0.002005,4.191156,...,4.036731,4.234547,1.089401,1.098462,2.902092,2.309390,0.098281,0.184798,2.847589,4.036731
3,0.085421,-0.015231,0.177250,0.070474,1.092853,1.038455,0.827731,0.834797,-0.285268,4.272981,...,4.092283,4.342240,1.005533,1.048950,2.902092,2.512851,-0.075352,0.154476,2.835485,4.092283
4,-0.369489,-1.014485,-1.715424,-0.353198,-0.850404,-0.998491,0.238732,0.659368,0.324838,4.237524,...,3.981180,4.388660,-1.010188,-0.970188,1.594299,5.519557,-0.246574,-0.384926,1.519379,3.981180


### Logistic regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,f1_score


In [9]:
scaler = StandardScaler()
X_train_tf = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index, columns=X_train.columns)
X_test_tf = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns=X_test.columns)

In [10]:
lr = LogisticRegression(solver='liblinear',max_iter=1000,penalty='l2',random_state=100)
lr.fit(X_train_tf,Y_train)
Y_train_pred = lr.predict(X_train_tf)

In [11]:
lr.fit(X_train,Y_train)
Y_train_pred = lr.predict(X_train)
print(np.unique(Y_train_pred))
Y_test_pred = lr.predict(X_test)
print(np.unique(Y_test_pred))


[1 2 3 4 5 6 7]
[1 2 3 4 5 6 7]


In [12]:
print(accuracy_score(Y_train,Y_train_pred))
Y_test_pred = lr.predict(X_test_tf)

0.6748015873015873


In [13]:
print(np.unique(Y_test_pred))
print(np.unique(Y_train_pred))
print(len(Y_test_pred))

[1 2 3 4 5 6 7]
[1 2 3 4 5 6 7]
565892


In [15]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['Cover_Type'] = Y_test_pred

In [16]:
sample_submission.to_csv('submission_lr.csv',index=False)

### Feature selection

In [17]:
from sklearn.feature_selection import SelectKBest, f_classif


In [18]:
feature_selector = SelectKBest(f_classif, k=50)
X_kbest = pd.DataFrame(feature_selector.fit_transform(X_train_tf, Y_train))

/Users/z002r2f/anaconda/lib/python3.6/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):
/Users/z002r2f/anaconda/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [20 28] are constant.
  UserWarning)
/Users/z002r2f/anaconda/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [19]:
lr.fit(X_kbest,Y_train)
Y_train_pred = lr.predict(X_kbest)
print(accuracy_score(Y_train,Y_train_pred))

0.6820767195767196


In [20]:
import sklearn.ensemble

### Decision Tree classifier

In [21]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train_tf,Y_train)
Y_train_pred_dt = dtc.predict(X_train_tf)

In [22]:
Y_train_pred_dt

array([5, 5, 2, ..., 3, 3, 3], dtype=int64)

### Random Forest Classifier

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [24]:
results_rf = pd.DataFrame(data=None,columns=['n_estimator','criterion','max_depth',
                                            'accuracy_min','accuracy_max','accuracy_mean'])

In [27]:
N_estimators = [20,40,60,80]
Criterion = ['gini','entropy']
Max_depth = [20,30,40,50,60,70]
i=0
for n_estimator in N_estimators:
    for criterion in Criterion:
        for max_depth in Max_depth:
            results_rf.loc[i,'n_estimator'] = n_estimator
            results_rf.loc[i,'criterion'] = criterion
            results_rf.loc[i,'max_depth'] = max_depth
            r = cross_val_score(RandomForestClassifier(n_estimators=n_estimator,
                                                       criterion=criterion,
                                                      max_depth=max_depth,
                                                      random_state=100),
                               X=X_train_tf,y=Y_train,scoring='accuracy',cv=5,n_jobs=4)
            results_rf.loc[i,'accuracy_min'] = r.min()
            results_rf.loc[i,'accuracy_max'] = r.max()
            results_rf.loc[i,'accuracy_mean'] = r.mean()
            i = i+1

In [25]:
RandomForestClassifier?

In [28]:
results_rf

,n_estimator,criterion,max_depth,accuracy_min,accuracy_max,accuracy_mean
0,20,gini,20,0.768188,0.864418,0.798942
1,20,gini,30,0.768519,0.866733,0.798016
2,20,gini,40,0.767196,0.866071,0.796825
3,20,gini,50,0.767196,0.866071,0.796825
4,20,gini,60,0.767196,0.866071,0.796825
5,20,gini,70,0.767196,0.866071,0.796825
6,20,entropy,20,0.762897,0.871362,0.799272
7,20,entropy,30,0.768849,0.868717,0.801455
8,20,entropy,40,0.768849,0.868717,0.801786
9,20,entropy,50,0.768849,0.868717,0.801786


In [ ]:
results_rf.sort_values('accuracy_mean')

In [ ]:
results_rf=pd.DataFrame(None,columns=['n_estimator','max_features','max_depth',
                                            'accuracy_min','accuracy_max','accuracy_mean'])

N_estimators = [80,100,120]
Max_depth = [50,60,70,80]
Max_features = ["auto",'log2',None,20]
i=0
for n_estimator in N_estimators:
    for max_features in Max_features:
        for max_depth in Max_depth:
            results_rf.loc[i,'n_estimator'] = n_estimator
            results_rf.loc[i,'max_features'] = max_features
            results_rf.loc[i,'max_depth'] = max_depth
            r = cross_val_score(RandomForestClassifier(n_estimators=n_estimator,
                                                       criterion='entropy',
                                                      max_depth=max_depth,
                                                      max_features=max_features,
                                                      random_state=100),
                               X=X_train_tf,y=Y_train,scoring='accuracy',cv=5,n_jobs=4)
            results_rf.loc[i,'accuracy_min'] = r.min()
            results_rf.loc[i,'accuracy_max'] = r.max()
            results_rf.loc[i,'accuracy_mean'] = r.mean()
            i = i+1

In [ ]:
results_rf.sort_values('accuracy_mean')

Best hyperparameters

max_depth = 50;
criterion = entropy;
n_estimators = 120;
max_feature = 20

In [ ]:
# parameter that give best score so far on test - 0.77105
# n_estimators=150,max_depth=50,max_features=20,criterion='entropy',max_features=20
r1 = cross_val_score(RandomForestClassifier(n_estimators=150,
                                                       criterion='entropy',
                                                      max_depth=50,
                                                      max_features=20,
                                                      random_state=100),
                               X=X_train_tf,y=Y_train,scoring='accuracy',cv=5,n_jobs=4)
r1.mean()

### GridSearchCV test for Random forest

In [30]:
# using gridsearchcv to emulate the for first set of 'for loops' to get the best parameters
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier(random_state=100)
parameters = {'n_estimators':(20,40,60,80),
              'criterion':('gini','entropy'),
              'max_depth':(20,30,40,50,60,70)}
clf = GridSearchCV(rf, parameters, scoring='accuracy', n_jobs = 4, cv=5)
clf.fit(X_train_tf, Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=100,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'n_estimators': (20, 40, 60, 80), 'criterion': ('gini', 'entropy'), 'max_depth': (20, 30, 40, 50, 60, 70)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [43]:
clf.cv_results_

{'mean_fit_time': array([0.58208251, 1.13955073, 1.67806759, 2.16584625, 0.54197946,
        1.09525309, 1.72296124, 2.29720693, 0.54915304, 1.09028296,
        1.66201477, 2.28195949, 0.56694846, 1.08589978, 1.76115565,
        2.3111546 , 0.5909698 , 1.21043954, 1.83927598, 2.27788072,
        0.57887945, 1.12238064, 1.70313687, 2.32578416, 0.74433718,
        1.49357405, 2.36854906, 3.16012635, 0.76947103, 1.53204699,
        2.31243277, 3.27995839, 0.8020246 , 1.61329379, 2.23830166,
        2.96130109, 0.75103636, 1.57932491, 2.37938161, 3.13244491,
        0.7984179 , 1.59640884, 2.23949299, 2.95376282, 0.74937038,
        1.48591361, 2.26613517, 2.9514493 ]),
 'mean_score_time': array([0.01820488, 0.03020177, 0.03948402, 0.05102763, 0.01490617,
        0.02861333, 0.04250789, 0.05447278, 0.01492052, 0.02877688,
        0.04477615, 0.05782404, 0.01497493, 0.03078203, 0.04720154,
        0.05772619, 0.0190002 , 0.03263721, 0.04558978, 0.05806293,
        0.01573935, 0.02809544, 0.

In [45]:
accuracy_mean = clf.cv_results_['mean_test_score']
n_estimators = clf.cv_results_['param_n_estimators']
criterion = clf.cv_results_['param_criterion']
max_depth = clf.cv_results_['param_max_depth']
print('Best Parameters',clf.best_params_,'\n')
print('Best accuracy score',clf.best_score_,'\n')
results = pd.DataFrame(data={'n_estimators':n_estimators,'criterion':criterion,
                             'max_depth':max_depth,'accuracy_mean':accuracy_mean})
results[['n_estimators','criterion','max_depth','accuracy_mean']]

Best Parameters {'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 80} 

Best accuracy score 0.8126984126984127 



,n_estimators,criterion,max_depth,accuracy_mean
0,20,gini,20,0.798942
1,40,gini,20,0.805622
2,60,gini,20,0.808135
3,80,gini,20,0.808862
4,20,gini,30,0.798016
5,40,gini,30,0.806349
6,60,gini,30,0.808466
7,80,gini,30,0.811111
8,20,gini,40,0.796825
9,40,gini,40,0.806481


### AdaBoost

In [42]:
from sklearn.ensemble import AdaBoostClassifier

In [102]:
AdaBoostClassifier?

In [101]:
ada = AdaBoostClassifier(n_estimators=10,learning_rate=1,random_state=100)

In [99]:
ada.fit(X_train_tf,Y_train)
Y_train_pred_ada = ada.predict(X_train_tf)
accuracy_score(Y_train,Y_train_pred_ada)

0.5089285714285714